In [4]:
pip install ucimlrepo

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from ucimlrepo import fetch_ucirepo

# Fetch dataset
student_performance = fetch_ucirepo(id=320)

# Extract features and targets
X = student_performance.data.features
y = student_performance.data.targets

# Convert features and targets to pandas DataFrames
X_df = pd.DataFrame(X, columns=student_performance.metadata.features)
y_df = pd.DataFrame(y, columns=student_performance.metadata.targets)

# Concatenate features and targets into a single DataFrame
student_df = pd.concat([X_df, y_df], axis=1)

# Display metadata
print(student_performance.metadata)

# Display variable information
print(student_performance.variables)


{'uci_id': 320, 'name': 'Student Performance', 'repository_url': 'https://archive.ics.uci.edu/dataset/320/student+performance', 'data_url': 'https://archive.ics.uci.edu/static/public/320/data.csv', 'abstract': 'Predict student performance in secondary education (high school). ', 'area': 'Social Science', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 649, 'num_features': 30, 'feature_types': ['Integer'], 'demographics': ['Sex', 'Age', 'Other', 'Education Level', 'Occupation'], 'target_col': ['G1', 'G2', 'G3'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Fri Jan 05 2024', 'dataset_doi': '10.24432/C5TG7T', 'creators': ['Paulo Cortez'], 'intro_paper': {'title': 'Using data mining to predict secondary school student performance', 'authors': 'P. Cortez, A. M. G. Silva', 'published_in': 'Proceedings of 5th Annual Future Business Technology Conference', 'year'

In [43]:
student_df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [48]:
student_df["G3"] = student_df["G3"].astype(int)
student_df["pass_fail"] = (student_df["G3"] >= 10).astype(int)


In [53]:
student_df = student_df.drop(columns = "G3")

In [54]:
student_df = pd.get_dummies(student_df)

In [56]:
X = student_df.drop(columns = "pass_fail", axis = 1)
y= student_df["pass_fail"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [59]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
features = X_train_scaled.shape[1]

nodes_layer1 = 100

nodes_layer2 = 80

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units = nodes_layer1, input_dim = features, activation = "relu"))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units = nodes_layer2, activation = "relu"))

# Output layer

nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               5900      
                                                                 
 dense_1 (Dense)             (None, 80)                8080      
                                                                 
 dense_2 (Dense)             (None, 1)                 81        
                                                                 
Total params: 14061 (54.93 KB)
Trainable params: 14061 (54.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [62]:
fitted_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
16/16 [==============================] - 1s 3ms/step - loss: 0.5493 - accuracy: 0.7572
Epoch 2/100
16/16 [==============================] - 0s 4ms/step - loss: 0.3363 - accuracy: 0.8519
Epoch 3/100
16/16 [==============================] - 0s 2ms/step - loss: 0.2637 - accuracy: 0.8827
Epoch 4/100
16/16 [==============================] - 0s 2ms/step - loss: 0.2169 - accuracy: 0.8992
Epoch 5/100
16/16 [==============================] - 0s 2ms/step - loss: 0.1798 - accuracy: 0.9280
Epoch 6/100
16/16 [==============================] - 0s 2ms/step - loss: 0.1496 - accuracy: 0.9547
Epoch 7/100
16/16 [==============================] - 0s 2ms/step - loss: 0.1193 - accuracy: 0.9733
Epoch 8/100
16/16 [==============================] - 0s 2ms/step - loss: 0.0954 - accuracy: 0.9815
Epoch 9/100
16/16 [==============================] - 0s 2ms/step - loss: 0.0783 - accuracy: 0.9918
Epoch 10/100
16/16 [==============================] - 0s 2ms/step - loss: 0.0633 - accuracy: 0.9897
Epoch 11/

In [63]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6/6 - 0s - loss: 0.5630 - accuracy: 0.8834 - 183ms/epoch - 30ms/step
Loss: 0.5629849433898926, Accuracy: 0.8834356069564819
